# 프롬프트 템플릿 작성

In [4]:
from dotenv import load_dotenv
import os

load_dotenv('.env') 
api_key = os.environ["OPENAI_API_KEY"]

In [5]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o", api_key=api_key)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1099dbc80>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10a931b50>, root_client=<openai.OpenAI object at 0x1099190a0>, root_async_client=<openai.AsyncOpenAI object at 0x109a3dc70>, model_name='gpt-3.5-turbo-0125', model_kwargs={}, openai_api_key=SecretStr('**********'))

# 템플릿 객체 사용

In [6]:
from langchain.prompts import PromptTemplate

# template 객체 생성 및 구성 정의
template = PromptTemplate.from_template(
    "{city}에서 {adjective} {topic}을 알려주세요.")

# template.format()으로 템플릿에 값을 대입
prompt = template.format(city="서울", 
                        adjective="가장 유명한",
                        topic="맛집"
                         )
print(prompt)

서울에서 가장 유명한 맛집을 알려주세요.


In [7]:
# llm에 질문 요청
response = llm.invoke(prompt)


In [ ]:
response

AIMessage(content='서울에서 가장 유명한 맛집 중 하나는 고궁입니다. 고궁은 전통적인 한식을 현대적으로 재해석한 요리를 맛볼 수 있는 곳으로 유명합니다. 특히 한옥 안에서 마실 수 있는 전통주와 함께 즐기는 한식요리는 많은 이들로부터 사랑받고 있습니다. 만약 전통적이고 정통적인 한식을 경험하고 싶다면 고궁을 방문해보시는 것을 추천드립니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 168, 'prompt_tokens': 28, 'total_tokens': 196, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BgQHMNXehhtWteJleLvl5MxesldTE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ac6b097e-fd81-4f35-becd-bb2c835e8e3f-0', usage_metadata={'input_tokens': 28, 'output_tokens': 168, 'total_tokens': 196, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
print(response.content)

서울에서 가장 유명한 맛집 중 하나는 고궁입니다. 고궁은 전통적인 한식을 현대적으로 재해석한 요리를 맛볼 수 있는 곳으로 유명합니다. 특히 한옥 안에서 마실 수 있는 전통주와 함께 즐기는 한식요리는 많은 이들로부터 사랑받고 있습니다. 만약 전통적이고 정통적인 한식을 경험하고 싶다면 고궁을 방문해보시는 것을 추천드립니다.


In [10]:
# template 객체를 JSON 파일로 저장
template.save("template.json")

# Chat Prompt Template

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    api_key=api_key,
    temperature=0.0,
    max_tokens=1000
)

chat_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 베테랑 영행 가이드로, 고객 최적의 맞춤 여행 일정 수립을 도와줍니다."),
    ("human", "여름에 {place}로 {travel_type}을 가려고 합니다."),
    ("ai", "제가 바로 {place} 전문 여행 가이드입니다. 어떤 도움이 필요하신가요?"),
    ("human", " 다음 질문에 답해주세요: {questions}")
])

prompt = chat_template.format_messages(
    when="여름",
    place="동남아",
    travel_type="가족여행",
    questions=[
        "여름에 동남아로 가족여행을 가면 어떤 것을 준비해야 할까요?",
        "동남아에서 가족과 함께 할 수 있는 활동은 무엇이 있을까요?",
        "가족과 함께 동남아에서 추천할 만한 음식은 무엇인가요?",
        "해당 지역의 여행지에 여름에 가족이 즐길만한 5군데를 추천해 주세요."
    ]
)

In [11]:
response = llm.invoke(prompt)

In [12]:
print(response.content)

서울에서 가장 유명한 맛집 중 하나로는 강남역 근처에 위치한 "초당골 순두부"가 유명합니다. 이곳은 전통적인 한국 음식 중 하나인 순두부찌게를 맛볼 수 있는 곳으로, 매콤하고 감칠맛 나는 순두부찌게를 즐길 수 있습니다. 또한, 푸짐한 양과 신선한 재료로 만들어지는 메뉴들이 많아 지역 주민 뿐만 아니라 관광객들에게도 인기가 많은 맛집입니다. 혹시 강남일대를 방문하게 된다면 추천해보고 싶은 곳 중 하나입니다.


# prompt pipeline 구성 방법

In [14]:
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

prompt_1 = PromptTemplate.from_template("당신은 {place} 베테랑 여행 가이드입니다. ")
prompt_2 = PromptTemplate.from_template("고객 최적의 {place} {travel} 일정 수립을 도와줍니다.")

final_template = """
{prompt_1}
{prompt_2}
"""

final_prompt = PromptTemplate.from_template(final_template)
final_prompt

PromptTemplate(input_variables=['prompt_1', 'prompt_2'], input_types={}, partial_variables={}, template='\n{prompt_1}\n{prompt_2}\n')

# prompt pipeline 구성 방법

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

prompt_1 = PromptTemplate.from_template("당신은 {place} 베테랑 여행 가이드입니다. ")
prompt_2 = PromptTemplate.from_template("고객 최적의 {place} {travel} 일정 수립을 도와줍니다.")

final_template = """
{prompt_1}
{prompt_2}
"""
#  최종적으로 완성될 프롬프트가 어떤 구조와 순서를 가질지 정의하는 **'빈 껍데기' 또는 '틀'**입니다.
# final_prompt 자체는 {place}나 {travel}과 같은 초기 변수를 전혀 모릅니다. 오직 {prompt_1}과 {prompt_2}라는 이름의 **'중간 결과물'**이 들어올 자리만 알고 있을뿐이다.
final_prompt = PromptTemplate.from_template(final_template)
final_prompt

PromptTemplate(input_variables=['prompt_1', 'prompt_2'], input_types={}, partial_variables={}, template='\n{prompt_1}\n{prompt_2}\n')

In [ ]:
# 프롬프트 파이프라인 설정 '내용물'을 만들고 '목차'와 연결하는 방법
pipeline_prompts = [
    ("prompt_1", prompt_1),
    ("prompt_2", prompt_2)
]
# 정의한 '빈 껍데기'의 각 부분을 어떤 재료와 방법으로 채울지 알려주는 **'실행 계획' 또는 '연결 고리'**입니다.
# PipelinePromptTemplate 생성
pipeline = PipelinePromptTemplate(
    final_prompt=final_prompt,
    pipeline_prompts=pipeline_prompts
)

/var/folders/19/rx9v5nfx6mvgj7s4njrn8yzm0000gn/T/ipykernel_31358/2911117945.py:8: LangChainDeprecationWarning: This class is deprecated. Please see the docstring below or at the link for a replacement option: https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.pipeline.PipelinePromptTemplate.html
  pipeline = PipelinePromptTemplate(


In [18]:
# 테스트 입력
input_data = {
    "place": "제주도",
    "travel": "3박 4일"
}

# 최종 프롬프트 텍스트 생성
formatted_prompt = pipeline.format(**input_data)

# 결과 출력
print("✅ 최종 생성된 프롬프트:\n")
print(formatted_prompt)

✅ 최종 생성된 프롬프트:


당신은 제주도 베테랑 여행 가이드입니다. 
고객 최적의 제주도 3박 4일 일정 수립을 도와줍니다.



# 부분 프롬프트

In [19]:
# 기본 템플릿 정의
prompt = PromptTemplate(template="{city}에서 유명한 {place}을 알려주세요.", 
                        input_variables=["place"],          # 사용자 입력이 필요한 변수
                        partial_variables={"city": "서울"})  # 초기값 입력

# input_variables 값 입력                        
print(prompt.format(place="한정식집"))

# partial_variables 값 변경
partial_prompt = prompt.partial(city="부산")

# input_variables 값 입력
print(partial_prompt.format(place="해운대"))

서울에서 유명한 한정식집을 알려주세요.
부산에서 유명한 해운대을 알려주세요.


# 프롬프트로 질문하기
- LCEL(LangChain Expression Language) 사용 예시

In [ ]:
from langchain_core.output_parsers import StrOutputParser
chain = partial_prompt | llm | StrOutputParser()

In [21]:
chain

PromptTemplate(input_variables=['place'], input_types={}, partial_variables={'city': '부산'}, template='{city}에서 유명한 {place}을 알려주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x1099dbc80>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10a931b50>, root_client=<openai.OpenAI object at 0x1099190a0>, root_async_client=<openai.AsyncOpenAI object at 0x109a3dc70>, model_name='gpt-3.5-turbo-0125', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [ ]:
# 전달값은 dict 형이여야 함. {"place": "해운대"}
chain_res = chain.invoke({"place": "해운대"})  

In [ ]:
chain_res

'부산의 해운대는 국내에서 가장 유명한 해변 중 하나입니다. 해운대는 깨끗한 백사장과 맑은 바다뿐만 아니라 다양한 관광 명소와 편의시설을 갖추고 있어 많은 관광객들이 찾는 곳입니다. 해변을 따라 걷는 해운대해수욕장 산책로, 해수욕장 근처에 위치한 센텀시티 백화점과 영화의 거리, 그리고 부산 아쿠아리움 등이 해운대를 대표하는 명소들 중 하나로 꼽힙니다. 특히 여름에는 많은 사람들이 모여 해수욕을 즐기기 때문에 더욱 활기차고 화려한 분위기를 느낄 수 있습니다.해운대는 교통도 편리하여 부산을 방문하는 관광객들에게 인기 있는 지역 중 하나입니다.'